# Train an IntegratedML model on Marketing Campaign Dataset
## Use JDBC to connect to InterSystems IRIS database
This Notebook demonstrates:
- Using the JayDeBeApi Python library to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict marketing campaign responses
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### 1. Set environment variables, if necessary

### 2. Get jdbc connection and cursor

In [1]:

import jaydebeapi
url = "jdbc:IRIS://irisimlsvr:51773/USER" #"jdbc:IRIS://172.17.0.1:8091/USER"
driver = 'com.intersystems.jdbc.IRISDriver'
user = "SUPERUSER"
password = "SYS"
#libx = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18"
#jarfile = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18/intersystems-jdbc-3.0.0.jar"
jarfile = "./intersystems-jdbc-3.1.0.jar"

In [2]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

java.sql.SQLException: java.sql.SQLException: [InterSystems IRIS JDBC] Communication link failure: Connection refused (Connection refused)

### 3. specify the source data table

In [ ]:
dataTable = 'SQLUser.Campaign'

### 4. Execute a query and display results in Pandas DataFrame

In [ ]:
import pandas as pd
from IPython.display import display

df = pd.read_sql("select TOP 20 * from %s" % dataTable, conn)
display(df)

In [ ]:
# Show number rows
df1 = pd.read_sql("SELECT COUNT(*) FROM %s" % dataTable, conn)
display(df1)

### 5. Make some views to split training and testing datasets

In [ ]:
# Small training set view
curs.execute("CREATE VIEW SQLUser.CampaignTrainingSmall AS SELECT * FROM SQLUser.Campaign WHERE ID<3001")
# Small prediction set
curs.execute("CREATE VIEW SQLUser.CampaignPredictSmall AS SELECT * FROM SQLUser.Campaign WHERE ID>47000")

### 6. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

In [ ]:
curs.execute("CREATE MODEL CampaignModel PREDICTING (RESPONSE) FROM CampaignTrainingSmall")

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [ ]:
curs.execute("TRAIN MODEL CampaignModel")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [ ]:
df3 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", conn)
display(df3)

### 7. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [ ]:
df4 = pd.read_sql("SELECT PREDICT(CampaignModel) AS PredictedResponse, \
                  response AS ActualResponse FROM CampaignPredictSmall", conn)
display(df4)

### 8. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [ ]:
curs.execute("VALIDATE MODEL CampaignModel FROM SQLUser.CampaignPredictSmall")
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", conn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)